## Mục lục:
* [I. Giới thiệu Recurrent Neural Network (RNN) và sự cần thiết của thuật toán Backpropagation through time (BPTT)](#section-i)
    * [1. Neural Network](#section-1)
        * [1.1. Giới thiệu](#section-1-1)
        * [1.2. Hạn chế của Neural Network truyền thống với dữ liệu tuần tự theo thời gian](#section-1-2)
    * [2. Recurrent Neural Network (RNN)](#section-2)
        * [2.1. Dữ liệu tuần tự theo thời gian](#section-2-1)
        * [2.2. Tổng quan về Recurrent Neural Network (RNN)](#section-2-2)
    * [3. Vấn đề Backpropagation trong Recurrent Neural Network](#section-3)
        * [3.1 Thuật toán Backpropagation hoạt động không tốt trên Recurrent Neural Network](#section-3-1)
        * [3.2 Sự cần thiết của Backpropagation Through Time (BPTT)](#section-3-2)

# I. Giới thiệu Recurrent Neural Network (RNN) và sự cần thiết của thuật toán Backpropagation through time (BPTT) <a id="section-i"></a>

# 1. Neural Network <a id="section-1"></a>

## 1.1. Giới thiệu <a id="section-1-1"></a>

Neural Network là một mô hình học máy mô phỏng hoạt động tương tự hệ thần kinh của con người. Neural Network bao gồm các lớp neuron liên kết với nhau bằng các trọng số, với cơ chế hoạt động:

- Nhận đầu vào (input).
- Tính toán tổng có trọng số + bias.
- Áp dụng hàm kích hoạt (activation function).
- Xuất ra giá trị dự đoán (output).

Trong quá trình huấn luyện, thuật toán **Backpropagation** được sử dụng để cập nhật trọng số của mạng, sao cho đầu ra tiệm cận giá trị mong muốn.

Ví dụ mô hình đơn giản:

- Input → Hidden Layer → Output Layer

<!-- <center><img src="images/neural-network.png"/></center> -->

<center>
<figure>
<img src="images/neural-network.png" style="width:50%">
<figcaption align = "center"> Mô hình Neural Network đơn giản </figcaption>
Nguồn: <a href="https://www.cloudflare.com/learning/ai/what-is-neural-network/" target="_blank">Cloudflare. What is a Neural Network?</a>
</figure>
</center>

## 1.2. Hạn chế của Neural Network truyền thống với dữ liệu tuần tự theo thời gian <a id="section-1-2"></a>

Neural Network truyền thống chỉ có thể xử lý thông tin theo một chiều từ Input → Output. Công thức chung:
$$
\mathbf{h}^{(l)} = \sigma(\mathbf{W}^{(l)} \mathbf{h}^{(l-1)} + \mathbf{b}^{(l)})
$$

Trong đó:
- $\mathbf{h}^{(l)}$ là vector output tầng $l$.
- $\mathbf{W}^{(l)}$, $\mathbf{b}^{(l)}$ là ma trận trọng số và bias.
- $\sigma$ là hàm kích hoạt (ReLU, tanh, sigmoid...).

Chính vì thông tin chỉ đi được một chiều mà không thể quay lại (do không có vòng lặp), các input tại các node không thể nhớ thông tin nào từ trước. Bên cạnh đó, nếu xử lý nhiều input, thì các input này cũng được xử lý độc lập và không biết giá trị trước sau. Cụ thể:

- Giả sử với chuỗi input:
$$
\mathbf{x}_1, \mathbf{x}_2, \mathbf{x}_3, \ldots, \mathbf{x}_T
$$

- Với Neural Network truyền thống:
$$
\mathbf{y}_t = f(\mathbf{x}_t)
$$

- Nhưng thực tế cần xử lý tuần tự:
$$
\mathbf{y}_t = f(\mathbf{x}_1, \mathbf{x}_2, \ldots, \mathbf{x}_t)
$$


Trong thực tế, rất nhiều bài toán trong thực tế đòi hỏi mô hình phải "nhớ" được các dữ liệu trong quá khứ, liên kết chúng lại và dự đoán giá trị output. Chẳng hạn như các bài toán sau đều không thể chỉ xử lý từng input đơn lẻ mà cần xem xét chuỗi các input liên tiếp:
- **Dịch máy (Machine Translation):** Nghĩa của một từ phụ thuộc vào các từ trước đó.
- **Dự đoán giá cổ phiếu:** Giá hiện tại bị ảnh hưởng bởi xu hướng giá trước đó.
- **Nhận diện giọng nói:** Âm thanh ở thời điểm hiện tại phụ thuộc ngữ cảnh âm thanh trước đó.
- **Chatbot hội thoại:** Câu trả lời hiện tại phải dựa trên lịch sử đoạn hội thoại.

Tóm lại, Neural Network truyền thống được thiết kế để xử lý các dữ liệu đầu vào độc lập, không có mối liên kết theo thời gian giữa các bước. Mô hình sẽ hoạt động hiệu quả khi dữ liệu đầu vào đã hoàn chỉnh và không mang tính chuỗi như trong các bài toán phân loại, hồi quy. Tuy nhiên, với dữ liệu tuần tự, phụ thuộc vào các thông tin trước đó, Neural Network hoàn toàn không có cơ chế ghi nhớ trạng thái hay ngữ cảnh. Chính vì thế, Recurrent Neural Networks (RNN) ra đời, với cơ chế hidden state được cập nhật liên tục qua mỗi bước thời gian, cho phép mô hình có thể lưu trữ và khai thác thông tin từ quá khứ, phục vụ cho việc xử lý chuỗi dữ liệu một cách tự nhiên và hiệu quả hơn.

# 2. Recurrent Neural Network (RNN) <a id="section-2"></a>
## 2.1. Dữ liệu tuần tự theo thời gian <a id="section-2-1"></a>

Trong nhiều bài toán thực tế, dữ liệu không chỉ đơn thuần độc lập mà còn mang bản chất tuần tự. Tức là, giá trị tại thời điểm hiện tại không tồn tại riêng lẻ, mà chịu ảnh hưởng bởi những dữ liệu xảy ra trước đó. Hiểu một cách đơn giản, để đưa ra quyết định ở thời điểm hiện tại, mô hình cần phải xem xét không chỉ dữ liệu hiện tại mà còn cả lịch sử các sự kiện trước đó.

Chẳng hạn về dữ liệu tuần tự trong xử lý ngôn ngữ tự nhiên (NLP), khi đọc một câu văn, việc hiểu nghĩa của một từ phụ thuộc rất nhiều vào các từ đi trước. Ví dụ:
  - Câu: `"Hôm nay trời rất ___."`
    - Nếu các từ trước đó là `"Hôm nay trời rất"`, thì từ tiếp theo hợp lý có thể là `"đẹp"`, `"nắng"`, `"mát"`,...
    - Nếu chỉ nhìn từ `"rất"`, sẽ không thể đoán chính xác được ý nghĩa mong muốn.


Để có thể xử lý loại dữ liệu trên, một mô hình cần có các đặc điểm sau:

- **Tiếp nhận một đầu vào tại mỗi thời điểm**:  
   - Ví dụ: một từ trong câu, một ảnh từ camera, một giá trị cảm biến đo được.

- **Cập nhật trạng thái ẩn (hidden state)**:  
   - Trạng thái ẩn đóng vai trò là bộ nhớ, lưu giữ những thông tin quan trọng từ các thời điểm trước. 
   - Hidden state được cập nhật dựa trên đầu vào hiện tại kết hợp với trạng thái trước đó.

- **Sinh ra đầu ra tương ứng**:  
   - Output tại thời điểm hiện tại có thể phụ thuộc không chỉ vào input hiện tại mà còn phụ thuộc vào toàn bộ lịch sử các input trước.

## 2.2. Tổng quan về Recurrent Neural Network (RNN) <a id="section-2-2"></a>

Recurrent Neural Network (RNN) được thiết kế để xử lý dữ liệu tuần tự bằng cách giới thiệu trạng thái ẩn (hidden state) để lưu trữ thông tin về các input trước đó.

Tại mỗi bước thời gian $t$, RNN thực hiện các bước sau:
- Nhận đầu vào $\mathbf{X}_t$,
- Kết hợp đầu vào hiện tại $\mathbf{X}_t$ và trạng thái ẩn từ bước trước $\mathbf{H}_{t-1}$,
- Cập nhật trạng thái ẩn $\mathbf{H}_t$,
- Sinh ra đầu ra $\mathbf{O}_t$ dựa trên trạng thái ẩn $\mathbf{H}_t$.

Cập nhật trạng thái ẩn:
$$
\mathbf{H}_t = \varphi(\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh} + \mathbf{b}_h)
$$

Trong đó:
- $\mathbf{X}_t \in \mathbb{R}^{n \times d}$: Input tại thời điểm $t$ (vector hoặc batch).
- $\mathbf{H}_t \in \mathbb{R}^{n \times h}$: Hidden state tại thời điểm $t$.
- $\mathbf{W}_{xh} \in \mathbb{R}^{d \times h}$: Trọng số từ input đến hidden.
- $\mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$: Trọng số từ hidden trước tới hidden hiện tại.
- $\mathbf{b}_h \in \mathbb{R}^{1 \times h}$: Bias cho hidden state.
- $\varphi$: Hàm kích hoạt (thường là tanh hoặc ReLU).

Output sinh ra:
$$
\mathbf{O}_t = \mathbf{H}_t \mathbf{W}_{hq} + \mathbf{b}_q
$$

Trong đó:
- $\mathbf{O}_t \in \mathbb{R}^{n \times q}$: Output tại thời điểm $t$.
- $\mathbf{W}_{hq} \in \mathbb{R}^{h \times q}$: Trọng số từ hidden đến output.
- $\mathbf{b}_q \in \mathbb{R}^{1 \times q}$: Bias cho output.

Lưu ý rằng, với mô hình ở các bước thời gian khác nhau, trọng số và bias là giống nhau. Chính vì thế, khi số lượng bước thời gian tăng lên, mô hình không cần thêm tham số mới, đảm bảo kích thước mô hình cố định bất kể độ dài của chuỗi đầu vào.

<center>
<figure>
<img src="images/rnn.png" style="width:50%">
<figcaption align = "center"> Ví dụ về Recurrent Neural Network đơn giản </figcaption>
Nguồn: <a href="https://towardsdatascience.com/a-brief-introduction-to-recurrent-neural-networks-638f64a61ff4/" target="_blank">Malik, A. A Brief Introduction to Recurrent Neural Networks</a>
</figure>
</center>



# 3. Vấn đề Backpropagation trong Recurrent Neural Network <a id="section-3"></a>

Trong Neural Network truyền thống, quá trình học sử dụng thuật toán backpropagation đơn giản, chỉ lan truyền lỗi ngược qua từng lớp, từ đầu ra về đầu vào. Thêm nữa, các trọng số cũng riêng biệt ở mỗi lớp, dữ liệu được xử lý độc lập, không có yếu tố thời gian. Tuy nhiên, trong mạng RNN, vì mô hình xử lý dữ liệu tuần tự theo thời gian, và trọng số được chia sẻ qua các bước thời gian nên quá trình tính toán gradient phải lan truyền ngược lại không chỉ qua các lớp mà còn xuyên suốt qua các bước thời gian. Phương pháp lan truyền lỗi ngược này được gọi là **Backpropagation Through Time (BPTT)**.

## 3.1 Thuật toán Backpropagation hoạt động không tốt trên Recurrent Neural Network <a id="section-3-1"></a>

Khi áp dụng trực tiếp backpropagation cho RNN, xuất hiện hai vấn đề lớn do tính chất của chuỗi thời gian:

**(1) Vanishing Gradient**
- Khi gradient được truyền ngược qua rất nhiều bước thời gian, mỗi bước đều nhân với ma trận trọng số và đạo hàm của hàm kích hoạt.
- Nếu giá trị của đạo hàm hoặc giá trị của trọng số nhỏ hơn 1, việc nhân liên tục qua nhiều bước sẽ làm cho gradient giảm dần về 0.
- Hậu quả:
  - Các trọng số liên quan đến các bước thời gian xa hầu như không được cập nhật.
  - Mô hình không học được mối quan hệ dài hạn trong dữ liệu (long-term dependencies).
- Ví dụ:
  - Khi dịch một câu quá dài, mô hình không thể nhớ những từ đầu tiên, dẫn đến bản dịch mất ngữ cảnh.

**(2) Exploding Gradient**
- Ngược lại, nếu giá trị của đạo hàm hoặc giá trị của trọng số lớn hơn 1, việc nhân liên tục qua nhiều bước sẽ làm cho gradient tăng cực nhanh.
- Gradient trở nên rất lớn, dẫn đến:
  - Quá trình cập nhật trọng số trở nên mất kiểm soát,
  - Quá trình huấn luyện dao động mạnh, thậm chí gây lỗi overflow hoặc NaN.
- Ví dụ:
  - Mô hình đưa ra các dự đoán vô nghĩa, nhảy lung tung, hoặc không hội tụ trong quá trình huấn luyện.


## 3.2 Sự cần thiết của Backpropagation Through Time (BPTT) <a id="section-3-2"></a>

Trong quá trình mô hình được huấn luyện từ dữ liệu tuần tự, tại mỗi thời điểm $t$, mô hình không chỉ cần xử lý dữ liệu đầu vào mới $\mathbf{X}_t$, mà còn phải cập nhật trạng thái ẩn dựa trên thông tin hiện tại và trạng thái tích lũy từ các bước thời gian trước đó. Việc duy trì và cập nhật trạng thái ẩn liên tục theo thời gian là yếu tố then chốt để mô hình có thể học được quan hệ dài hạn của chuỗi dữ liệu. Khi mô hình cần cập nhật bộ tham số, việc tính toán gradient được thực hiện xuyên suốt chuỗi thời gian thay vì chỉ qua các lớp như trong mạng feedforward truyền thống. Điều này nhằm đảm bảo rằng thông tin từ quá khứ không bị "quên" và mối quan hệ theo thời gian được học một cách hiệu quả. Tuy nhiên, việc lan truyền gradient qua nhiều bước thời gian liên tiếp lại gây ra những vấn đề nghiêm trọng là vanishing gradient và exploding gradient như đã đề cập phía trên. Từ đó, khiến mô hình bị mất những thông tin đầu tiên khi input quá dài, hoặc mô hình dự đoán vô nghĩa. Chính vì vậy, việc huấn luyện RNN yêu cầu một kỹ thuật đặc biệt được gọi là Backpropagation Through Time (BPTT), cho phép mở rộng mạng RNN theo thời gian và tính toán gradient chính xác trên toàn bộ chuỗi.

# Tài liệu tham khảo
- Cloudflare. (n.d.). What is a neural network? Cloudflare. Retrieved April 25, 2025, from https://www.cloudflare.com/learning/ai/what-is-neural-network/
- D2L Vietnam. (n.d.). Mạng nơ-ron hồi quy (Recurrent Neural Networks). Dive into Deep Learning (D2L). Retrieved April 25, 2025, from https://d2l.aivivn.com/chapter_recurrent-neural-networks/rnn_vn.html
- Malik, A. (2022, December 26). A brief introduction to recurrent neural networks. Towards Data Science. Retrieved April 27, 2025, from https://towardsdatascience.com/a-brief-introduction-to-recurrent-neural-networks-638f64a61ff4/
- StatQuest with Josh Starmer. (2022, July 11). Recurrent Neural Networks (RNNs), Clearly Explained!!!. YouTube. https://youtu.be/AsNTP8Kwu80?si=P1-F2Ypu-MJC-VqT
- Wikipedia contributors. (n.d.). Vanishing gradient problem. Wikipedia. Retrieved April 27, 2025, from https://en.wikipedia.org/wiki/Vanishing_gradient_problem